In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time
from bs4 import BeautifulSoup
import threading
from concurrent.futures import ThreadPoolExecutor

In [2]:
# Declare the lists for each feature to be scraped
repo_url = []
repo_watches = []
repo_sponsors = []
repo_open_issues = []
repo_closed_issues = []
repo_labels = []
repo_milestones = []
repo_open_prs = []
repo_closed_prs = []
lock = threading.Lock()

In [3]:
# Potential Scrapes:
# ../issues => Open/closed issues, # of labels, # of milestones
# ../pulls
# ../actions => # of workflow runs
# ../pulse => Active pr and Active issues
# ../network/dependencies
# Sponsered
# Watches
# 
# Ones in Bash Script:
# Commits
# Tags/Releases
# Branches
# Languages
# Contributors

# Headless mode for Chrome
chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-renderer-backgrounding")
chrome_options.add_argument("--disable-background-timer-throttling")
chrome_options.add_argument("--disable-backgrounding-occluded-windows")
chrome_options.add_argument("--disable-client-side-phishing-detection")
chrome_options.add_argument("--disable-crash-reporter")
chrome_options.add_argument("--disable-oopr-debug-crash-dump")
chrome_options.add_argument("--no-crash-upload")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-low-res-tiling")
chrome_options.add_argument("--log-level=3")
chrome_options.add_argument("--silent")
chrome_options.add_argument("--blink-settings=imagesEnabled=false")

def scrape_page(project_url):

    project_features = []
    
    print(project_url)
    # Add url to list
    project_features.append(project_url)
    
    # Get the OWNER/REPO
    project = project_url[19:]
    print(project)

    # Set up Web Driver
    driver = webdriver.Chrome(options=chrome_options)

    driver.get(project_url)

    # Get number of watches and sponsors
    # Wait for the document to be in 'complete' state
    WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.TAG_NAME, 'body'))
    )
    
    # Parse HTML
    # Get number of watches and sponsered?
    html = driver.page_source
    soup = BeautifulSoup(html,"html.parser")
    
    num_watches = soup.find(href=f"/{project}/watchers").find("strong").text
    
    creator = project.split('/')[0]
    sponsored = "Yes" if soup.find(href=f"/sponsors/{creator}") != None else "No"

    project_features.append(num_watches)
    project_features.append(sponsored)
    
    # Issues
    #thread = threading.Thread(name=watch_sponsors,target=issues(project_url, project, driver))
    #thread.start()
    issue_url = project_url + "/issues"
    driver.get(issue_url)

    # Wait for the document to be in 'complete' state
    WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.TAG_NAME, 'body'))
    )
    
    html = driver.page_source
    soup = BeautifulSoup(html,"html.parser")

    open_issues = None if soup.find(href=f"/{project}/issues?q=is%3Aopen+is%3Aissue") == None else soup.find(href=f"/{project}/issues?q=is%3Aopen+is%3Aissue").text.split()[0]
    closed_issues = None if soup.find(href=f"/{project}/issues?q=is%3Aissue+is%3Aclosed") == None else soup.find(href=f"/{project}/issues?q=is%3Aissue+is%3Aclosed").text.split()[0]
    num_labels = None if soup.find(href=f"/{project}/labels") == None else soup.find(href=f"/{project}/labels").find("span").text
    num_milestones = None if soup.find(href=f"/{project}/milestones") == None else soup.find(href=f"/{project}/milestones").find("span").text


    project_features.append(open_issues)
    project_features.append(closed_issues)
    project_features.append(num_labels)
    project_features.append(num_milestones)

    # Pull Requests
    pull_url = project_url + "/pulls"
    driver.get(pull_url)

    # Wait for the document to be in 'complete' state
    WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.TAG_NAME, 'body'))
    )
        
    html = driver.page_source
    soup = BeautifulSoup(html,"html.parser")

    print(f"Getting prs: {project}")
    open_prs = soup.find(href=f"/{project}/pulls?q=is%3Aopen+is%3Apr").text.split()[0]
    closed_prs = soup.find(href=f"/{project}/pulls?q=is%3Apr+is%3Aclosed").text.split()[0]

    print(f"Getting prs: {project} Open={open_prs} Closed={closed_prs}")
    
    project_features.append(open_prs)
    project_features.append(closed_prs)
    
    # Clean close the Web Session and window(s)
    driver.quit()

    return project_features

In [4]:
#project_list = pd.read_excel('project_5000Up.xlsx')

In [5]:
#project_list = project_list['Project URL'].tolist()
#roject_list[:10]
#project_list = ["https://github.com/spring-attic/spring-mvc-showcase","https://github.com/twbs/bootstrap", "https://github.com/freeCodeCamp/freeCodeCamp", "https://github.com/geekyutao/Inpaint-Anything", "https://github.com/raspberrypi/firmware", "https://github.com/negomi/react-burger-menu", "https://github.com/yyhsong/iDataV", "https://github.com/yipianfengye/android-zxingLibrary", "https://github.com/nikic/FastRoute", "https://github.com/vercel/platforms", "https://github.com/thinkingjimmy/Learning-Prompt"]
project_list = ["https://github.com/spring-attic/spring-mvc-showcase","https://github.com/twbs/bootstrap"]




In [6]:
#df = pd.read_excel('projects.xlsx') # can also index sheet by name or fetch all sheets
#project_list = df['Project'].tolist()
#project_list = ["https://github.com/twbs/bootstrap", "https://github.com/freeCodeCamp/freeCodeCamp"]
#for p in project_list[:10]:
#    scrape_page(p)
projects = []
for project in project_list:
    result = scrape_page(project)
    projects.append(result)
#with ThreadPoolExecutor() as p:
    #features = p.map(scrape_page, project_list)

    #for f in features:
        #projects.append(f)

https://github.com/spring-attic/spring-mvc-showcase
spring-attic/spring-mvc-showcase
Getting prs: spring-attic/spring-mvc-showcase
Getting prs: spring-attic/spring-mvc-showcase Open=9 Closed=34
https://github.com/twbs/bootstrap
twbs/bootstrap
Getting prs: twbs/bootstrap
Getting prs: twbs/bootstrap Open=137 Closed=14,900


In [7]:
#print(len(repo_url))
#rint(len(repo_watches))
#print(len(repo_sponsors))
#print(len(repo_open_issues))
#print(len(repo_closed_issues))
#print(len(repo_labels))
#print(len(repo_milestones))
#print(len(repo_open_prs))
#print(len(repo_closed_prs))

In [8]:

projects_df = pd.DataFrame(projects, columns=['Project URL', 'Number of Watches', 'Sponsored','Open Issues','Closed Issues','Number of Labels','Number of Milestones','Open Pull Requests','Closed Pull Requests'])


In [9]:
projects_df

,Project URL,Number of Watches,Sponsored,Open Issues,Closed Issues,Number of Labels,Number of Milestones,Open Pull Requests,Closed Pull Requests
0,https://github.com/spring-attic/spring-mvc-sho...,528,No,5,38,0,0,9,34
1,https://github.com/twbs/bootstrap,6.8k,Yes,395,"21,902",60,0,137,"14,900"


try:
    with pd.ExcelWriter(
        "project_HTMLfeatures.xlsx",
        mode="a",
        engine="openpyxl",
        if_sheet_exists="overlay",
    ) as writer:
         projects_df.to_excel(writer,sheet_name="Sheet1", startrow=writer.sheets["Sheet1"].max_row, index = False,header= False)
except FileNotFoundError:
    projects_df.to_excel("project_HTMLfeatures.xlsx", index=False)